In [0]:
%matplotlib inline
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [85]:
df = pd.read_csv('../data/new_data_only.csv', names = ['x_acc', 'y_acc', 'z_acc'])
label_df = pd.read_csv('../data/new_labels_only.csv', header = None)
print(df.head())
print(label_df.head())

      x_acc     y_acc     z_acc
0  0.496614  0.507404  0.664175
1  0.593133  0.544373  0.639089
2  0.583067  0.535586  0.626219
3  0.556202  0.507976  0.593481
4  0.556658  0.492937  0.577615
     0    1    2    3
0  0.0  0.0  1.0  0.0
1  0.0  0.0  1.0  0.0
2  0.0  0.0  1.0  0.0
3  0.0  0.0  1.0  0.0
4  0.0  0.0  1.0  0.0


In [0]:
traindf = df[ : int(0.7 * len(df))]
testdf = df[int(0.85 * len(df)) : ]
valdf = df[int(0.7 * len(df)) : int(0.85 * len(df))]
trainlabel = label_df[ : int(0.7 * len(df))]
testlabel = label_df[int(0.85 * len(df)) : ]
vallabel = label_df[int(0.7 * len(df)) : int(0.85 * len(df))]

train_gen = TimeseriesGenerator(traindf.values, trainlabel.values, length = 100, batch_size = 16, stride = 100)
test_gen = TimeseriesGenerator(testdf.values, testlabel.values, length = 100, batch_size = 1, stride = 100)
val_gen = TimeseriesGenerator(valdf.values, vallabel.values, length = 100, batch_size = 1, stride = 100)

In [0]:
n_input = 100
n_features = 3
batch_size = 1
model = Sequential()
model.add(keras.layers.GRU(80, activation = 'relu', input_shape = (n_input, n_features), return_sequences = True))
model.add(keras.layers.GRU(60, activation = 'relu', return_sequences = True))
model.add(keras.layers.GRU(40, activation = 'relu'))
model.add(Dense(4, activation = 'softmax'))
adam = keras.optimizers.Adam(lr = 5e-4)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [102]:
history = model.fit_generator(train_gen, validation_data = val_gen, epochs = 50, verbose = 1)

Epoch 1/50
24/24 [==============================] - 32s 1s/step - loss: 1.3406 - acc: 0.4566 - val_loss: 1.3054 - val_acc: 0.4177
Epoch 2/50
24/24 [==============================] - 19s 788ms/step - loss: 1.2345 - acc: 0.4988 - val_loss: 1.2352 - val_acc: 0.4177
Epoch 3/50
24/24 [==============================] - 19s 783ms/step - loss: 1.1928 - acc: 0.4988 - val_loss: 1.2189 - val_acc: 0.4177
Epoch 4/50
24/24 [==============================] - 19s 780ms/step - loss: 1.1910 - acc: 0.4988 - val_loss: 1.2181 - val_acc: 0.4177
Epoch 5/50
24/24 [==============================] - 19s 778ms/step - loss: 1.1909 - acc: 0.4988 - val_loss: 1.2150 - val_acc: 0.4177
Epoch 6/50
24/24 [==============================] - 19s 783ms/step - loss: 1.1903 - acc: 0.4988 - val_loss: 1.2154 - val_acc: 0.4177
Epoch 7/50
24/24 [==============================] - 19s 777ms/step - loss: 1.1898 - acc: 0.4988 - val_loss: 1.2135 - val_acc: 0.4177
Epoch 8/50
24/24 [==============================] - 19s 791ms/step - los

In [0]:
model.evaluate_generator(test_gen)

In [0]:
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()